In [107]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

#Ad-Soyad: İpek Erdoğan
#Besin Değerlerinden Sınıflandırma projesi seçildi.

In [131]:
#path for data
train_path = "train2.csv"
test_path = "test2.csv"

def include_more_dot(strr):
    count=0
    for k in range(len(strr)):
        if (strr[k]=="."):
            count=count+1
    if (count>1):
        strr = strr.split('.')
        strr = strr[0] + strr[1] + strr[2]
    return strr

def editing (input1):
    for i in range(len(input1)):
        for k in range(32):
            if (input1[i][k] == "Mikrogram"):
                input1[i][k] = 0.001
            if (input1[i][k] == "Gram"):
                input1[i][k] = 1
            if (type(input1[i][k])==str):
                input1[i][k] = include_more_dot(input1[i][k])
            input1[i][k] = float(input1[i][k])
    return input1

#load data
def loadData(train_path,test_path):
    #convert data to array
    train_data = np.asarray(pd.read_csv(train_path))
    test_data = np.asarray(pd.read_csv(test_path))
    #split last column for Y (class label)
    x_train = train_data[:,:32]
    y_train1 = train_data[:,32]
    
    x_train = editing(x_train)
    test_data = editing(test_data)
    return x_train,y_train1,test_data
#load data with given paths
x_train, y_train1,test_data = loadData(train_path,test_path)

# def preprocessing(inp):
#     #Principal Component Analysys
#     #PCA
#     #reduction to 3 features
#     pca = KernelPCA(n_components=3,kernel='linear')
#     p_comp_train = pca.fit_transform(inp)
#     principalData = pd.DataFrame(data = p_comp_train)

#     # Feature Scaling
#     scaler = StandardScaler()
#     scaler.fit(principalData)
#     return scaler.transform(principalData)

# #PCA to train data
# #x_train1 = preprocessing(x_train)
# #apply same transformation to test data
# #X_test = preprocessing(test_data)

In [140]:
def trainModel(x_train1,y_train1):

    #Tried these classifiers and choose the best one of them
    classifiers = [
        KNeighborsClassifier(n_neighbors=100,weights='distance'),
        DecisionTreeClassifier(criterion="entropy"),
        RandomForestClassifier(n_estimators=100),
        AdaBoostClassifier(n_estimators=100,learning_rate=0.001),
        LogisticRegression(solver='lbfgs', max_iter=10000)]


    #initialization to store best model and best train/test data
    i=0
    acc = 0.0
    accuracies = np.zeros(len(classifiers))
    bestTrainx = x_train1
    bestTrainy = y_train1
    bestAccuracy = 0.0
    bestCross =0.0
    bestModel = KNeighborsClassifier()
    #for every classifier in the list
    for clf in classifiers:
        model = clf
        name = clf.__class__.__name__
        #try 15 different train/test group
        #this train data is choosen randomly every time (shuffle = True)
        for k in range(15):#cross validation
            #split TRAİN data to train and test
            x_train, x_test, y_train, y_test = train_test_split(x_train1, y_train1,shuffle=True,)
            model.fit(x_train,y_train)
            prediction = model.predict(x_test)
            #measure the performance
            score = cross_val_score(model,x_test,y_test,cv=3)
            score = score.mean()
            acc = accuracy_score(y_test,prediction)
            #if accuracy is good store it in bestModel, bestTrainx and bestTrainy
            #for training later
            if acc > bestAccuracy:
                print("best changed to,",name,"\n")
                bestname = name
                bestAccuracy = acc
                bestCross = score
                bestTrainx = x_train
                bestTrainy = y_train
                bestModel = model
            if acc == bestAccuracy:
                if score > bestCross:
                    print("best changed to,", name, "\n")
                    bestAccuracy = acc
                    bestCross = score
                    bestTrainx = x_train
                    bestTrainy = y_train
                    bestModel = model
                    bestname = name

        #just to see success of each classifier
        print(name," accuracy:", accuracy_score(y_test, prediction))
        print("cross validation accuracy: ", score)
        print("\n")
        print("best accuracy: ",bestAccuracy, bestname,bestCross)
        accuracies[i]=accuracy_score(y_test, prediction)#accuracy scor for all classifiers in an array
        i += 1
    #best model is choosen, train it with it
    bestModel.fit(bestTrainx,bestTrainy)
    #return trained model
    return bestModel
    #to see each classifier at once, nothing that important
    #print("all accuracies: ",accuracies)

In [141]:
bestModel = trainModel(x_train,y_train1)

def predict(model,Xtst):
    Result = model.predict(Xtst)
    return Result

def writeOutput(result):
    submission = pd.DataFrame({'Class': result})
    #submission.index += 1
    print("submission done\n")
    submission.to_csv('Result.csv',index_label='ID')

writeOutput(predict(bestModel,test_data))

best changed to, KNeighborsClassifier 

best changed to, KNeighborsClassifier 

best changed to, KNeighborsClassifier 

best changed to, KNeighborsClassifier 

KNeighborsClassifier  accuracy: 0.5664
cross validation accuracy:  0.5199975463133357


best accuracy:  0.5808 KNeighborsClassifier 0.49924089068825905
best changed to, DecisionTreeClassifier 

best changed to, DecisionTreeClassifier 

best changed to, DecisionTreeClassifier 

best changed to, DecisionTreeClassifier 

best changed to, DecisionTreeClassifier 

best changed to, DecisionTreeClassifier 

best changed to, DecisionTreeClassifier 

DecisionTreeClassifier  accuracy: 0.7008
cross validation accuracy:  0.6959038768249295


best accuracy:  0.7408 DecisionTreeClassifier 0.6688213102686786
best changed to, RandomForestClassifier 

best changed to, RandomForestClassifier 

best changed to, RandomForestClassifier 

RandomForestClassifier  accuracy: 0.7888
cross validation accuracy:  0.7952015090172986


best accuracy:  0.8112 